In [83]:
import os
import pandas as pd
import numpy as np
jq = "GZ"

In [84]:
resColumns = ["std", "mean", "q_50", "q_90", "q_99", "q_999", "q_9999"]
model_index = ["all_delay", "XIAO_DU", "sub_du", "ALARM", "APP_MANAGEMENT", "GREETING", "PHONE", 
               "ROBOROCK_CLEAN_FUNCTION", "ROBOROCK_CLEAN_FUNCTION_MULTI", "ROBOROCK_OTHER_FUNCTION", "ROBOROCßK_PARAMETER_CONTROL",
               "ROBOROCK_SCHEDULE_MANAGEMENT", "SPACE", "SYS_COMMAND", "SYS_PARAMETER", "SYS_PERIPHERALS", "SYS_PROFILE", "TIME", "Unit"]
resData = []
dates = ["2024-05-18", "2024-05-19"]
filePath = []
root = '/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log'
for date in dates:
    fileDir=f"data/version2_0/{date}/queryByresId"
    fileList = os.listdir(root+'/'+fileDir)
    for file in  fileList:
        if file.endswith(".csv"):
            filePath.append(f"{root}/{fileDir}/{file}")
filePath

['/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/version2_0/2024-05-18/queryByresId/es_origin_data_12:00:00.csv',
 '/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/version2_0/2024-05-18/queryByresId/es_origin_data_06:00:00.csv',
 '/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/version2_0/2024-05-18/queryByresId/es_origin_data_00:00:00.csv',
 '/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/version2_0/2024-05-18/queryByresId/es_origin_data_18:00:00.csv',
 '/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/version2_0/2024-05-19/queryByresId/es_origin_data_12:00:00.csv',
 '/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/version2_0/2024-05-19/queryByresId/es_origin_data_06:00:00.csv',
 '/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/version2_0/2024-05-19/queryByresId/es_origin_data_00:00:00.csv',
 '/Users/liangzonglin/workplace/project/baidu/bc

In [85]:
dataList = []
for file in filePath:
    df = pd.read_csv(file)[["ts","region","seg","reqId","domain","subXD","req"]]
    df.dropna(axis=0, subset=["reqId", "ts", "region"], how='any', inplace=True)
    df = df[df['region'].isin(["BD", "SZ", "GZ"])]
    dataList.append(df)
datadf = pd.concat(dataList, axis=0, ignore_index=True)
datadf.head()

,ts,region,seg,reqId,domain,subXD,req
0,2024-05-18 12:00:52.914,SZ,IOT2ASR,d343a73a07204cc1b2ec6f1ac904819a,NaN,False,NaN
1,2024-05-18 12:00:52.915,SZ,NaN,d343a73a07204cc1b2ec6f1ac904819a,NaN,False,NaN
2,2024-05-18 12:00:52.238,SZ,IOT2DU,2cac00156a044e7e86139a3ae87fadba,NaN,False,NaN
3,2024-05-18 12:00:52.242,SZ,NaN,2cac00156a044e7e86139a3ae87fadba,NaN,False,NaN
4,2024-05-18 12:00:52.242,SZ,NaN,2cac00156a044e7e86139a3ae87fadba,NaN,False,NaN


In [86]:
datadf = datadf[datadf["region"] == jq]

In [87]:
len(datadf)

3102252

In [88]:
# 计算总延时,去重复
datadf.drop_duplicates(inplace=True)
print(f"数据数量：{str(len(datadf))}")
IOT2ASR_df = datadf[datadf["seg"]=="IOT2ASR"][["reqId", "ts"]]
ASR2IOT_df = datadf[datadf["seg"]=="ASR2IOT"][["reqId", "ts"]]
all_time_df = pd.merge(ASR2IOT_df, IOT2ASR_df, on=["reqId"], how="inner")
all_time_df.head()

数据数量：1745179


,reqId,ts_x,ts_y
0,13e8596203c8426a96ab5955e2b74b53,2024-05-18 12:00:54.387,2024-05-18 12:00:54.392
1,affead14a5964d6990daa004240c5869,2024-05-18 12:00:54.098,2024-05-18 12:00:54.156
2,7da20dd4f13042278252972d7f911736,2024-05-18 12:00:54.688,2024-05-18 12:00:55.032
3,2797c41545374322b2455def374a8c1e,2024-05-18 12:00:50.500,2024-05-18 12:00:50.542
4,bfabcb4fd2a04a70a01b6ea96b8b4b4f,2024-05-18 12:00:38.812,2024-05-18 12:00:38.817


In [89]:
all_time_df["ts_x"] = pd.to_datetime(all_time_df["ts_x"])
all_time_df["ts_y"] = pd.to_datetime(all_time_df["ts_y"])
all_time_df['duration'] = (all_time_df["ts_y"] - all_time_df["ts_x"]).dt.total_seconds()*1000
all_time_df.head()

,reqId,ts_x,ts_y,duration
0,13e8596203c8426a96ab5955e2b74b53,2024-05-18 12:00:54.387,2024-05-18 12:00:54.392,5.0
1,affead14a5964d6990daa004240c5869,2024-05-18 12:00:54.098,2024-05-18 12:00:54.156,58.0
2,7da20dd4f13042278252972d7f911736,2024-05-18 12:00:54.688,2024-05-18 12:00:55.032,344.0
3,2797c41545374322b2455def374a8c1e,2024-05-18 12:00:50.500,2024-05-18 12:00:50.542,42.0
4,bfabcb4fd2a04a70a01b6ea96b8b4b4f,2024-05-18 12:00:38.812,2024-05-18 12:00:38.817,5.0


In [90]:
def compute_q(serise):
    mean = round(serise.mean(), 1)
    std = round(serise.std(), 1)
    q_5 = round(serise.quantile(0.5, interpolation="lower"), 1)
    q_9 = round(serise.quantile(0.9, interpolation="lower"), 1)
    q_99 = round(serise.quantile(0.99, interpolation="lower"), 1)
    q_999 = round(serise.quantile(0.999, interpolation="lower"), 1)
    q_9999 = round(serise.quantile(0.9999, interpolation="lower"), 1)
    print(f"std: {std} \t mean: {mean}\t q_50: {q_5} \t q_90: {q_9} \t q_99: {q_99} \t q_999: {q_999}\t q_9999: {q_9999}")
    return (std, mean, q_5, q_9, q_99, q_999, q_9999)
resData.append(compute_q(all_time_df["duration"]))

std: 297.3 	 mean: 125.9	 q_50: 7.0 	 q_90: 347.0 	 q_99: 1921.0 	 q_999: 2255.0	 q_9999: 3006.0


In [91]:
datadf["seg"].value_counts()

ASR2IOT        75241
IOT2ASR        75240
IOT2DU         21619
DU2IOT         21617
IOT2MOJIV2      1220
MOJIV22IOT      1220
ASR2CORRECT        7
CORRECT2ASR        7
Name: seg, dtype: int64

In [92]:
# 计算xiaodu
IOT2DU_df = datadf[datadf["seg"]=="IOT2DU"][["reqId", "ts"]]
DU2IOT_df = datadf[datadf["seg"]=="DU2IOT"][["reqId", "ts"]]
du_time_df = pd.merge(IOT2DU_df, DU2IOT_df, on=["reqId"], how="inner")
du_time_df["ts_x"] = pd.to_datetime(du_time_df["ts_x"])
du_time_df["ts_y"] = pd.to_datetime(du_time_df["ts_y"])
du_time_df['duration'] = (du_time_df["ts_y"] - du_time_df["ts_x"]).dt.total_seconds()*1000
resData.append(compute_q(du_time_df["duration"]))

std: 607.1 	 mean: 822.1	 q_50: 486.0 	 q_90: 1854.0 	 q_99: 2200.0 	 q_999: 3029.0	 q_9999: 3231.0


In [93]:
du_time_df.head()

,reqId,ts_x,ts_y,duration
0,40e72a53302842bbbb2f6ddf1100002a,2024-05-18 12:00:44.211,2024-05-18 12:00:44.650,439.0
1,455a02fe24fc47ec92c7edbb0273661c,2024-05-18 12:00:36.156,2024-05-18 12:00:36.588,432.0
2,9439711dd1464046b893608b0ab3164c,2024-05-18 12:00:37.706,2024-05-18 12:00:38.089,383.0
3,889417c22fff43ccb8422c027c4d40d5,2024-05-18 12:00:44.796,2024-05-18 12:00:45.127,331.0
4,24831240a2264095b8e8d6be46b5e804,2024-05-18 12:00:44.653,2024-05-18 12:00:44.985,332.0


In [94]:
# sub_du_time = pd.merge(du_time_df[["reqId", "ts_y", "duration", "XiaoDu_y"]], all_time_df[["reqId", "ts_y", "duration", "XiaoDu"]], on="reqId", how="inner")
# sub_du_time.head()
# 计算减去xiaodu的耗时
sub_bool = datadf[datadf["seg"].notna()][["subXD", "reqId"]]
sub_req_id = sub_bool[sub_bool["subXD"]]["reqId"]
# sub_req_id = datadf[datadfsub_bool]["reqId"]
sub_du_data = all_time_df[~all_time_df["reqId"].isin(sub_req_id)]

In [95]:
sub_du_data.sort_values("duration", ascending=False)[:10]

,reqId,ts_x,ts_y,duration
48947,2f06ee5882b14b0194931f41c450729e,2024-05-19 17:07:11.215,2024-05-19 17:07:11.651,436.0
41727,8647a9c18b694244b28c94a7a9422c4a,2024-05-19 13:36:49.597,2024-05-19 13:36:50.030,433.0
14413,07f475331fed489e8c241783aa14654b,2024-05-18 08:15:41.512,2024-05-18 08:15:41.927,415.0
17266,0b9fb15ca45443aa98da8197843d5ea9,2024-05-18 09:58:06.093,2024-05-18 09:58:06.489,396.0
35707,2f0b6213959b49fdb7e209454f9a4b9e,2024-05-18 22:19:56.241,2024-05-18 22:19:56.630,389.0
15331,264ade473fe244afa604ed911461f710,2024-05-18 09:13:28.901,2024-05-18 09:13:29.289,388.0
23260,b6e79285574f49878ed0580ba74e144c,2024-05-18 11:19:45.772,2024-05-18 11:19:46.154,382.0
26059,9e26c46b519b41309c2cbdc80ce9ee90,2024-05-18 18:40:09.838,2024-05-18 18:40:10.206,368.0
13631,c6427b4e4a764cd8af89f7e3a1b40810,2024-05-18 08:30:05.935,2024-05-18 08:30:06.299,364.0
13550,889432a26d114b609a5663a8dc917e4e,2024-05-18 07:54:59.307,2024-05-18 07:54:59.654,347.0


In [96]:
sub_du_top10 = sub_du_data.sort_values("duration", ascending=False)[:10]
sub_du_top10.columns = ["请求id", "输入时间", "输出时间", "总延时(ms)"]
sub_du_top10.to_csv(f"stone_{jq}_dura_top10.csv", index=False)

In [97]:
sub_du_top10

,请求id,输入时间,输出时间,总延时(ms)
48947,2f06ee5882b14b0194931f41c450729e,2024-05-19 17:07:11.215,2024-05-19 17:07:11.651,436.0
41727,8647a9c18b694244b28c94a7a9422c4a,2024-05-19 13:36:49.597,2024-05-19 13:36:50.030,433.0
14413,07f475331fed489e8c241783aa14654b,2024-05-18 08:15:41.512,2024-05-18 08:15:41.927,415.0
17266,0b9fb15ca45443aa98da8197843d5ea9,2024-05-18 09:58:06.093,2024-05-18 09:58:06.489,396.0
35707,2f0b6213959b49fdb7e209454f9a4b9e,2024-05-18 22:19:56.241,2024-05-18 22:19:56.630,389.0
15331,264ade473fe244afa604ed911461f710,2024-05-18 09:13:28.901,2024-05-18 09:13:29.289,388.0
23260,b6e79285574f49878ed0580ba74e144c,2024-05-18 11:19:45.772,2024-05-18 11:19:46.154,382.0
26059,9e26c46b519b41309c2cbdc80ce9ee90,2024-05-18 18:40:09.838,2024-05-18 18:40:10.206,368.0
13631,c6427b4e4a764cd8af89f7e3a1b40810,2024-05-18 08:30:05.935,2024-05-18 08:30:06.299,364.0
13550,889432a26d114b609a5663a8dc917e4e,2024-05-18 07:54:59.307,2024-05-18 07:54:59.654,347.0


In [98]:
resData.append(compute_q(sub_du_data["duration"]))

std: 59.8 	 mean: 40.0	 q_50: 6.0 	 q_90: 152.0 	 q_99: 239.0 	 q_999: 290.0	 q_9999: 368.0


In [99]:
datadf['domain'].value_counts()

Unit                             44816
PHONE                            42374
SPACE                            42374
SYS_COMMAND                      42374
ROBOROCK_CLEAN_FUNCTION_MULTI    42374
GREETING                         42374
ROBOROCK_PARAMETER_CONTROL       42374
ROBOROCK_SCHEDULE_MANAGEMENT     42374
ROBOROCK_OTHER_FUNCTION          42374
ROBOROCK_CLEAN_FUNCTION          42374
SYS_PERIPHERALS                  42374
SYS_PARAMETER                    42374
SYS_PROFILE                      42374
ALARM                            42374
TIME                             42374
APP_MANAGEMENT                   42374
XiaoDu                           19465
Name: domain, dtype: int64

In [100]:
datadf["req"] = datadf["req"].str.lower()
def deal(name):
    name_df = df[df["domain"]==name]
    request_df = name_df[name_df["req"]=="request"][["reqId", "ts"]]
    response_df = name_df[name_df["req"]=="response"][["reqId", "ts"]]
    time_df = pd.merge(request_df, response_df, on=["reqId"], how="inner")
    time_df["ts_x"] = pd.to_datetime(time_df["ts_x"])
    time_df["ts_y"] = pd.to_datetime(time_df["ts_y"])
    sub_time = (time_df["ts_y"] - time_df["ts_x"]).dt.total_seconds()*1000
    return compute_q(sub_time)
# model_type = datadf['domain'].value_counts().index
for model in model_index[3:]:
    print(model)
    resData.append(deal(model))

ALARM
std: 5.5 	 mean: 15.1	 q_50: 14.0 	 q_90: 22.0 	 q_99: 32.0 	 q_999: 52.0	 q_9999: 77.0
APP_MANAGEMENT
std: 5.4 	 mean: 14.0	 q_50: 13.0 	 q_90: 21.0 	 q_99: 31.0 	 q_999: 45.0	 q_9999: 60.0
GREETING
std: 5.8 	 mean: 14.4	 q_50: 14.0 	 q_90: 21.0 	 q_99: 31.0 	 q_999: 50.0	 q_9999: 92.0
PHONE
std: 5.4 	 mean: 14.3	 q_50: 13.0 	 q_90: 21.0 	 q_99: 32.0 	 q_999: 51.0	 q_9999: 65.0
ROBOROCK_CLEAN_FUNCTION
std: 4.9 	 mean: 14.0	 q_50: 12.0 	 q_90: 21.0 	 q_99: 31.0 	 q_999: 45.0	 q_9999: 51.0
ROBOROCK_CLEAN_FUNCTION_MULTI
std: 6.3 	 mean: 22.6	 q_50: 21.0 	 q_90: 30.0 	 q_99: 44.0 	 q_999: 65.0	 q_9999: 93.0
ROBOROCK_OTHER_FUNCTION
std: 4.9 	 mean: 14.1	 q_50: 12.0 	 q_90: 21.0 	 q_99: 30.0 	 q_999: 41.0	 q_9999: 52.0
ROBOROCK_PARAMETER_CONTROL
std: 4.7 	 mean: 13.0	 q_50: 11.0 	 q_90: 20.0 	 q_99: 29.0 	 q_999: 40.0	 q_9999: 51.0
ROBOROCK_SCHEDULE_MANAGEMENT
std: 5.0 	 mean: 16.1	 q_50: 14.0 	 q_90: 23.0 	 q_99: 34.0 	 q_999: 49.0	 q_9999: 60.0
SPACE
std: 5.3 	 mean: 14.0	 q_50: 12.

In [101]:
resdf = pd.DataFrame(resData, index=model_index, columns=resColumns)
resdf.head()

,std,mean,q_50,q_90,q_99,q_999,q_9999
all_delay,297.3,125.9,7.0,347.0,1921.0,2255.0,3006.0
XIAO_DU,607.1,822.1,486.0,1854.0,2200.0,3029.0,3231.0
sub_du,59.8,40.0,6.0,152.0,239.0,290.0,368.0
ALARM,5.5,15.1,14.0,22.0,32.0,52.0,77.0
APP_MANAGEMENT,5.4,14.0,13.0,21.0,31.0,45.0,60.0


In [102]:
resdf.to_csv(f"stone_{jq}_time.csv")